# Training on 1,2 testing on 3,4,5,6 

In [4]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv("final_dataset.csv")

# Set the input image size
img_width, img_height = 224, 224

# Set the batch size
batch_size = 32

# Split the dataset into training and testing sets
train_df = df[df["fitzpatrick_scale"].isin([1, 2])]
test_df = df[df["fitzpatrick_scale"].isin([3, 4, 5, 6])]

# Define the training and testing data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    #horizontal_flip=True,
    #validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    subset="training"
)
"""

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    subset="validation"
)
"""
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

# Build the model
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(len(train_generator.class_indices), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False
    
# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator
)

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(test_generator)
print("Testing accuracy: {:.2f}%".format(accuracy * 100))



Found 6913 validated image filenames belonging to 114 classes.
Found 5302 validated image filenames belonging to 114 classes.
Epoch 1/20
217/217 [==============================] - 1352s 6s/step - loss: 4.1030 - accuracy: 0.1052 - val_loss: 4.1647 - val_accuracy: 0.0894
Epoch 2/20
217/217 [==============================] - 1158s 5s/step - loss: 3.3899 - accuracy: 0.1975 - val_loss: 4.0081 - val_accuracy: 0.1309
Epoch 3/20
217/217 [==============================] - 1092s 5s/step - loss: 3.0011 - accuracy: 0.2698 - val_loss: 3.9864 - val_accuracy: 0.1318
Epoch 4/20
217/217 [==============================] - 1116s 5s/step - loss: 2.6529 - accuracy: 0.3385 - val_loss: 4.0466 - val_accuracy: 0.1439
Epoch 5/20
217/217 [==============================] - 1129s 5s/step - loss: 2.3619 - accuracy: 0.4039 - val_loss: 4.0573 - val_accuracy: 0.1577
Epoch 6/20
217/217 [==============================] - 1093s 5s/step - loss: 2.0891 - accuracy: 0.4635 - val_loss: 4.1279 - val_accuracy: 0.1680
Epoch 7/20

In [9]:
import numpy as np

# Load test set
test_df = pd.read_csv('final_dataset.csv')

# Create data generator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Predict labels for test set
y_pred = model.predict(test_generator)
predicted_labels = np.argmax(y_pred, axis=1)
actual_labels = test_generator.classes

# Map label indices to label names
label_map = {v: k for k, v in test_generator.class_indices.items()}
predicted_labels = [label_map[label] for label in predicted_labels]
actual_labels = [label_map[label] for label in actual_labels]

# Get fitzpatrick_scale values for test set
fitzpatrick_scale = test_df['fitzpatrick_scale']

# Combine the predicted labels, actual labels, and fitzpatrick_scale values into a DataFrame
results_df = pd.DataFrame({'predicted_label': predicted_labels, 'actual_label': actual_labels, 'fitzpatrick_scale': fitzpatrick_scale})

# Save the DataFrame to a CSV file
results_df.to_csv('results_densetnet_tr1n2.csv', index=False)

# Print individual accuracies for each fitzpatrick scale value
for scale in range(3, 7):
    scale_df = results_df[results_df['fitzpatrick_scale'] == scale]
    total = len(scale_df)
    correct = sum(scale_df['predicted_label'] == scale_df['actual_label'])
    accuracy = correct / total
    print(f"Fitzpatrick Scale {scale}: {accuracy:.2%} ({correct}/{total})")


Found 12215 validated image filenames belonging to 114 classes.
382/382 [==============================] - 6272s 16s/step
Fitzpatrick Scale 3: 1.09% (25/2295)
Fitzpatrick Scale 4: 1.01% (17/1689)
Fitzpatrick Scale 5: 0.90% (8/891)
Fitzpatrick Scale 6: 1.41% (6/427)


In [12]:
#UPDATED CODE
import pandas as pd

# read csv file into a pandas dataframe
df = pd.read_csv("results_densetnet_tr1n2.csv")

# compute total count of fitzpatrick_scale
total_count = df.shape[0]

print(total_count)

# compute count of correctly predicted values
correctly_predicted = df[df["actual_label"] == df["predicted_label"]].shape[0]
print(correctly_predicted)

answer = (correctly_predicted/total_count)*100

print(answer)

12215
168
1.3753581661891117


In [13]:
accuracy = 16.67
d_accuracy = 1.50
new = accuracy/d_accuracy

f=[1.09,1.01,0.90,1.41]
l = [None] * 4

for i in range(0,4):
    l[i]=f[i]*new
print(l)

[12.113533333333336, 11.224466666666668, 10.002000000000002, 15.669800000000002]
